# Generatiing .pb file for XOR neural network

This notebook includes code for generation of tensorflow .pb file for a simple model. The model will behave like an XOR valve, using two inputs, both of which can be either zero or one, and produce one output that will be zero if both inputs are identical, and one otherwise. In addition, since it will be a deep model, it will have two hidden layers: one with four neurons and the other with three neurons. You can change the number of hidden layers and the number of neurons they contain.

In [ ]:
import tflearn
import tensorflow as tf

In [ ]:
X = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]
 
Y = [
    [0],  # Desired output for inputs 0, 0
    [1],  # Desired output for inputs 0, 1
    [1],  # Desired output for inputs 1, 0
    [0]   # Desired output for inputs 1, 1
]

In [ ]:
weights = tflearn.initializations.uniform(minval = -1, maxval = 1)
# Input layer
net = tflearn.input_data(
        shape = [None, 2],
        name = 'my_input'
)
 
# Hidden layers
net = tflearn.fully_connected(net, 4,
        activation = 'sigmoid',
        weights_init = weights
)
net = tflearn.fully_connected(net, 3,
        activation = 'sigmoid',
        weights_init = weights
)
 
# Output layer
net = tflearn.fully_connected(net, 1,
        activation = 'sigmoid', 
        weights_init = weights,
        name = 'my_output'
)

In [ ]:
net = tflearn.regression(net,
        learning_rate = 2,
        optimizer = 'sgd',
        loss = 'mean_square'
)
model = tflearn.DNN(net)
model.fit(X, Y, 5000)

Check how our model trained.

In [15]:
print("1 XOR 0 = %f" % model.predict([[1,0]]).item(0))
print("1 XOR 1 = %f" % model.predict([[1,1]]).item(0))
print("0 XOR 1 = %f" % model.predict([[0,1]]).item(0))
print("0 XOR 0 = %f" % model.predict([[0,0]]).item(0))

1 XOR 0 = 0.980426
1 XOR 1 = 0.013819
0 XOR 1 = 0.988445
0 XOR 0 = 0.028224


Removing training information and saving trained model

In [ ]:
# Remove train ops
with net.graph.as_default():
    del tf.get_collection_ref(tf.GraphKeys.TRAIN_OPS)[:]
# Save the model
model.save('xor.tflearn')

### Freezing model
The process of freezing a model involves the conversion of all its variables into constants. In addition, the frozen model must be a single binary file that conforms to the serialization format of the Google Protocol Buffers.

In [1]:
with tf.Session() as session:
    my_saver = tf.train.import_meta_graph('xor.tflearn.meta')
    my_saver.restore(session, tf.train.latest_checkpoint('.'))
    # As its inputs, the function expects the current session, 
    # the definition of the graph of the current session,
    # a list containing the names of the output levels of the model.
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        session,
        session.graph_def,
        ['my_output/Sigmoid']
    )
    with open('frozen_model.pb', 'wb') as f:
        f.write(frozen_graph.SerializeToString())

NameError: name 'tf' is not defined